In [1]:
using LinearAlgebra, Plots, Printf, Latexify, LaTeXStrings, BenchmarkTools, ArnoldiMethod, SparseArrays
using MatrixMarket: mmread

In [2]:
function MGS_arnoldi(A, q1::Vector{Float64}, k::Int)
  n, _ = size(A)
  Q = zeros(Float64, (n, k + 1))
  q = zeros(Float64, n)
  H = zeros(Float64, (k + 1, k))
  w = zeros(Float64,n)
  Q[:, 1] = q1 ./ norm(q1)
  for j in 1:k
    w .= A * Q[:, j]        
    for i in 1:j
      q .= Q[:, i]
      H[i, j] = q'w
      w -= H[i, j] .* q
    end
    H[j + 1, j] = norm(w)
    Q[:, j + 1] = w ./ H[j + 1, j]
  end
  return Q, H
end;

In [3]:
function CGS2_arnoldi(A, q1::Vector{Float64}, k::Int)
  n, _ = size(A)
  Q = zeros(Float64, (n, k + 1))
  H = zeros(Float64, (k + 1, k))
  h = zeros(Float64, k + 1)
  w = zeros(Float64, n)
  Q[:, 1] = q1 ./ norm(q1)
  for j in 1:k
    w .= A * Q[:, j]
    h .= Q'w
    h[j+1:k+1] .= 0.
    H[:, j] = h
    w -= Q * h
    h .= Q'w
    h[j+1:k+1] .= 0.
    w -= Q * h
    H[j + 1, j] = norm(w)
    Q[:, j + 1] = w ./ H[j + 1, j]
  end
  return Q, H
end;

In [4]:
matrix_folder = "/home/venkovic/Dropbox/Git/matrix-market/";
A = mmread(matrix_folder * "G3_circuit.mtx");
n, _ = size(A);

In [5]:
k = 100;

In [6]:
Q, H = @time MGS_arnoldi(A, rand(n), k);
@printf "||Qjp1 * A * Qj - H|| = %.2E, ||A * Qj - Qjp1 * H|| = %.2E, ||I - Q'Q|| = %.2E" norm(Q'A * Q[:, 1:k] - H) norm(A * Q[:, 1:k] - Q * H) norm(I - Q'Q)

 69.157852 seconds (1.03 M allocations: 183.798 GiB, 7.47% gc time, 0.79% compilation time)
||Qjp1 * A * Qj - H|| = 3.06E-07, ||A * Qj - Qjp1 * H|| = 5.49E-11, ||I - Q'Q|| = 2.19E-11

In [7]:
Q, H = @time CGS2_arnoldi(A, rand(n), k);
@printf "||Qjp1 * A * Qj - H|| = %.2E, ||A * Qj - Qjp1 * H|| = %.2E, ||I - Q'Q|| = %.2E" norm(Q'A * Q[:, 1:k] - H) norm(A * Q[:, 1:k] - Q * H) norm(I - Q'Q)

 17.489222 seconds (788.68 k allocations: 9.537 GiB, 2.74% gc time, 1.19% compilation time)
||Qjp1 * A * Qj - H|| = 3.19E-10, ||A * Qj - Qjp1 * H|| = 1.03E-10, ||I - Q'Q|| = 1.54E-14

In [8]:
function RR_CGS2_arnoldi(A, q1::Vector{Float64}, k::Int, nev::Int, tol::Float64)
  n, _ = size(A)
  Q = zeros(Float64, (n, k + 1))
  H = zeros(Float64, (k + 1, k))
  h = zeros(Float64, k + 1)
  w = zeros(Float64, n)
  Q[:, 1] = q1 ./ norm(q1)
  j = 0
  res = 1.; vals = nothing; vecs = nothing; order = nothing
  while (maximum(res) > tol && j < k)
    j += 1
    w .= A * Q[:, j]
    h .= Q'w
    h[j+1:k+1] .= 0.
    H[:, j] = h
    w -= Q * h
    h .= Q'w
    h[j+1:k+1] .= 0.
    w -= Q * h
    H[j + 1, j] = norm(w)
    Q[:, j + 1] = w ./ H[j + 1, j]        
    vals, vecs = eigen(H[1:j, 1:j], sortby=norm)
    order = (j:-1:j-min(nev,j)+1)
    res = norm(H[j + 1, j]) .* norm.(vecs[j, order]) ./ norm.(vals[order])
    println(res)
  end
  return res, vals[order], Q[:, 1:j] * vecs[:, order]
end;

In [9]:
A = mmread(matrix_folder * "nxp1.mtx");
n, _ = size(A);

In [10]:
k = 100;
nev = 5;
tol = 1e-6;
res, vals, Y = RR_CGS2_arnoldi(A, rand(n), k, nev, tol);

[7.122190536983693]
[0.8213522929785088, 9.132070839382395]
[0.5090906451152992, 0.9879623558688576, 9.85342639352183]
[0.2273285199551234, 0.4130076592013465, 0.8011141775928895, 7.287252375293572]
[0.0696241098782475, 0.2601789558053935, 0.606765853883062, 1.07063419974055, 7.854856070851897]
[0.02013586829794183, 0.1780901942406374, 0.27429748430277373, 0.524671225477858, 1.3501215708298435]
[0.005564758513671639, 0.11108444163147926, 0.2112494665794819, 0.4192124328680065, 0.5246420362943479]
[0.003241196162871559, 0.21812661373313652, 0.32094412867700595, 0.2957351880150637, 0.49040313059309226]
[0.001529257320021997, 0.13255933323165336, 0.07110862903214481, 0.12807679626255883, 0.25674920629979]
[0.00047570169093799636, 0.05096215691801789, 0.044388988086093914, 0.1735011136480694, 0.27515939270303824]
[9.060420082827789e-5, 0.013461710384683696, 0.025131351611863285, 0.12178200953833454, 0.0966774419378679]
[2.480720137416568e-5, 0.005142123151920981, 0.01982634357677782, 0.110

In [11]:
res2 = [norm(A * Y[:, i] - vals[i] .* Y[:, i]) / norm(vals[i]) for i in 1:nev];
maximum(res - res2)

-2.7830326273224106e-19

In [12]:
decomp, hist = partialschur(A, nev=nev);
vals2, Y2 = partialeigen(decomp);

In [13]:
maximum(abs.(vals2[end:-1:1] .- vals))

2.566480361565482e-10

In [14]:
norm(abs.(Y'Y2[:, end:-1:1]) - I)

5.285318735527176e-10

In [15]:
vals

5-element Vector{Float64}:
 414.7385747346634
 341.53916763358933
 257.3852366214957
 257.18132297539614
 255.8853137892908

In [16]:
function HR_CGS2_arnoldi(A, q1::Vector{Float64}, k::Int, nev::Int, sigma::ComplexF64, tol::Float64)
  n, _ = size(A)
  Q = zeros(Float64, (n, k + 1))
  H = zeros(Float64, (k + 1, k))
  h = zeros(Float64, k + 1)
  w = zeros(Float64, n)
  Q[:, 1] = q1 ./ norm(q1)
  j = 0
  res = 1.; rho = nothing; vecs = nothing
  while (maximum(res) > tol && j < k)
    j += 1
    w .= A * Q[:, j]
    h .= Q'w
    h[j+1:k+1] .= 0.
    H[:, j] = h
    w -= Q * h
    h .= Q'w
    h[j+1:k+1] .= 0.
    w -= Q * h
    H[j + 1, j] = norm(w)
    Q[:, j + 1] = w ./ H[j + 1, j]
    e_j = zeros(Float64, j); e_j[j] = 1.
    f = (H[1:j, 1:j] - sigma * I)' \ e_j
    G = H[1:j, 1:j] + norm(H[j + 1, j])^2 .* f * e_j'
    vals, vecs = eigen(G)
    order = sortperm(norm.(vals .- sigma))[1:minimum([j, nev])]
    vals = vals[order]
    vecs = vecs[:, order]    
    rho = [vals[i] - norm(H[j + 1, j])^2 * vecs[:, i]'f * vecs[j, i] for i in 1:minimum([j, nev])]
    res = sqrt.(real.((vals .- rho) .* conj.(rho .- sigma)))
    res ./= norm.(rho)
    #hres = (vals .- rho) .* conj.(vals .- sigma)
    println(res)
  end
  return res, rho, Q[:, 1:j] * vecs
end;

In [17]:
k = 500
nev = 2;
sigma = 150. + 0im;
res, vals, Y = HR_CGS2_arnoldi(A, rand(n), k, nev, sigma, tol);

[7.1533009469765005]
[0.8347931699754206, 6.965511864238601]
[0.7447323401602826, 11.615621258471972]
[0.7446611957425694, 12.056859813018207]
[0.4692006024418052, 0.2463101153680702]
[0.476589199338655, 0.24935107210545898]
[0.30049014260577256, 0.1990881323736262]
[0.17231726425356142, 0.296512887787958]
[0.10711384644426265, 0.18744545315384703]
[0.10533840544451503, 0.18673476965799582]
[0.09014495867313536, 0.12925256634201995]
[0.08239838565996903, 0.1311511535107435]
[0.08011291640243635, 0.09841309920481343]
[0.06917905782274152, 0.1026973614329002]
[0.07000517057626393, 0.08594696905715747]
[0.05432009685716049, 0.09343606795679026]
[0.050892486360531404, 0.09658983212822907]
[0.05692375290614733, 0.09340886286268016]
[0.051532539473104606, 0.0937317567017631]
[0.05777397074788321, 0.08724884798578428]
[0.06801660562031975, 0.04959547451079063]
[0.06837289487047638, 0.05017278951585166]
[0.060732438752970665, 0.06410432784492559]
[0.05637157468950246, 0.06166851413804627]
[0.0

[0.0002833546622832024, 0.00016285081052397717]
[0.0002997321583162542, 0.00011780717572280366]
[0.0002936665467458017, 9.690739100957879e-5]
[0.0002869321927758688, 0.0038714340604262074]
[0.0002857743906743429, 0.0037263511059510176]
[0.000263234199881088, 0.002979441213930258]
[0.00022479006784642148, 0.0022511782469827675]
[0.000171099783184913, 0.0015470024733860192]
[0.00012948628765532865, 0.0011094395274281151]
[0.00012193004931349803, 0.0010351599497683609]
[0.00011755840818921474, 0.0009955689123188752]
[8.826840366766627e-5, 0.0007304143639519665]
[5.193045655917833e-5, 0.0004194933008109507]
[2.647924374800029e-5, 0.0002109257098790371]
[1.4055552574518284e-5, 0.00011108773378453284]
[9.000081506659507e-6, 7.070189145191475e-5]
[7.696539122417452e-6, 6.017877894108288e-5]
[7.696809546032461e-6, 6.0236009430897344e-5]
[6.5212041723962e-6, 5.1198738090588106e-5]
[5.067512481228536e-6, 3.964417947070777e-5]
[3.7536490170743595e-6, 2.9100994118685784e-5]
[2.4616513496084925e-6,

In [18]:
res2 = [norm(A * Y[:, i] - vals[i] .* Y[:, i]) / norm(vals[i]) for i in 1:nev];
maximum(res - res2)

4.581184905592847e-14

In [19]:
vals

2-element Vector{ComplexF64}:
 151.07427423364803 - 6.242451922324095e-15im
 151.37728530092386 - 7.396799549638819e-15im

In [20]:
res

2-element Vector{Float64}:
 9.525835318490645e-8
 6.913572547902072e-7

In [21]:
function lanczos(A, q1::Vector{Float64}, k::Int)
  n, _ = size(A)
  Q = zeros(Float64, (n, k + 1))
  v = zeros(Float64, n)
  T = zeros(Float64, k + 1, k + 1)
  Q[:, 1] = q1 ./ norm(q1)
  for j in 1:k
    v = A * Q[:, j]
    T[j, j] = Q[:, j]'v
    v = v - T[j, j] * Q[:, j]
    if (j > 1)
      v -= T[j, j - 1] * Q[:, j - 1]
    end
    T[j + 1, j] = norm(v); T[j, j + 1] = T[j + 1, j]
    Q[:, j + 1] = v ./ T[j + 1, j]
  end
  return Q, T[1:k+1, 1:k]
end;

In [22]:
A = mmread(matrix_folder * "Kuu.mtx");
n, _ = size(A);

In [23]:
k = 100;

In [24]:
Q, T = @time lanczos(A, rand(n), k);
@printf "||Qjp1 * A * Qj - T|| = %.2E, ||A * Qj - Qjp1 * T|| = %.2E, ||I - Q'Q|| = %.2E" norm(Q'A * Q[:, 1:k] - T) norm(A * Q[:, 1:k] - Q * T) norm(I - Q'Q)

  0.162612 seconds (83.75 k allocations: 63.608 MiB, 19.92% gc time, 67.10% compilation time)
||Qjp1 * A * Qj - T|| = 4.00E-09, ||A * Qj - Qjp1 * T|| = 2.44E-14, ||I - Q'Q|| = 8.05E-11

In [25]:
function RR_lanczos(A, q1::Vector{Float64}, k::Int, nev::Int, tol::Float64)
  n, _ = size(A)
  Q = zeros(Float64, (n, k + 1))
  v = zeros(Float64, n)
  T = zeros(Float64, k + 1, k + 1)
  vals = zeros(Float64, nev, k)
  Q[:, 1] = q1 ./ norm(q1)
  j = 0
  res = 1.; vecs = nothing; order = nothing
  while (maximum(res) > tol && j < k)
    j += 1
    v = A * Q[:, j]
    T[j, j] = Q[:, j]'v
    v = v - T[j, j] * Q[:, j]
    if (j > 1)
      v -= T[j, j - 1] * Q[:, j - 1]
    end
    T[j + 1, j] = norm(v); T[j, j + 1] = T[j + 1, j]
    Q[:, j + 1] = v ./ T[j + 1, j]
    _vals, vecs = eigen(T[1:j, 1:j], sortby=norm)
    order = (j:-1:j-min(nev,j)+1)
    vals[1:minimum([j, nev]), j] = _vals[order]
    res = norm(T[j + 1, j]) .* norm.(vecs[j, order]) ./ norm.(_vals[order])
    println(res)
  end
  return res, vals[:, 1:j], Q[:, 1:j] * vecs[:, order]
end;

In [26]:
k = 500;
nev = 5;
res, vals, Y = RR_lanczos(A, rand(n), k, nev, tol);

[2.450030640409883]
[0.40893319735789274, 3.712501946875074]
[0.24906849988960983, 0.5874787349662829, 5.050051524338161]
[0.13633059379195456, 0.25364878911316335, 0.5753301566995878, 5.772381561120496]
[0.0686593946661566, 0.18651708890302957, 0.43665009298749274, 0.76098318386086, 7.6256436365211036]
[0.07235369399741466, 0.2470931083058264, 0.23642075007202154, 0.39440787228262963, 0.725651427636171]
[0.05184587233127862, 0.1008610433514588, 0.10889908259247442, 0.28999668327961037, 0.47170605168756874]
[0.039856159814400306, 0.0748329028930517, 0.21009494313809457, 0.2821084092764323, 0.30320201429980725]
[0.03423419327117202, 0.07187121400376824, 0.18342042949959494, 0.10920536607917516, 0.2868812158569415]
[0.032666846863417094, 0.0733062642895, 0.11106869915535864, 0.13684748873042615, 0.2235679229161952]
[0.02055086515777001, 0.04439548927012044, 0.05705386548759976, 0.13227553388461083, 0.10916034410864009]
[0.02005001440102662, 0.046602092108928864, 0.06968896540798757, 0.21

[7.722349444205167e-6, 1.1620180321371332e-5, 0.0003274713824268297, 0.001169210060975942, 0.007841319482887165]
[6.373295736916944e-6, 9.925130973845685e-6, 0.0003036804317375635, 0.0011766397713488044, 0.007717545322504772]
[3.965535148783185e-6, 6.300235420425072e-6, 0.00020229880513534843, 0.0008239114950612496, 0.005269606081140454]
[3.5047610361928215e-6, 5.664640327415218e-6, 0.0001890427658414125, 0.0007993252733083961, 0.005038056481165301]
[2.738004100541008e-6, 4.512661662394474e-6, 0.000157109714441621, 0.0006910570549273278, 0.004309464186751172]
[1.938627475292364e-6, 3.2474961642021773e-6, 0.00011696753292451179, 0.0005302178858811817, 0.0032957627772621143]
[1.5671735706308952e-6, 2.662519202448715e-6, 9.862548223599796e-5, 0.00045745297499289145, 0.002852543775503908]
[1.5248019696506725e-6, 2.6444057787064276e-6, 0.00010196280888372023, 0.00048791898630276076, 0.003075604679500319]
[8.820145385956313e-7, 1.5586088089327677e-6, 6.235121137813989e-5, 0.00030698514697588

In [27]:
res2 = [norm(A * Y[:, i] - vals[i, end] .* Y[:, i]) / norm(vals[i, end]) for i in 1:nev];
maximum(res - res2)

1.1672503134646086e-17

In [28]:
vals[:, end]

5-element Vector{Float64}:
 54.082055996412535
 53.98165804506269
 53.815496452830146
 53.720402051506426
 53.622541454743214

In [29]:
function HR_lanczos(A, q1::Vector{Float64}, k::Int, nev::Int, sigma::Float64, tol::Float64)
  n, _ = size(A)
  Q = zeros(Float64, (n, k + 1))
  v = zeros(Float64, n)
  T = zeros(Float64, k + 1, k + 1)
  rhos = zeros(ComplexF64, nev, k)
  Q[:, 1] = q1 ./ norm(q1)
  j = 0
  res = 1.; vecs = nothing; order = nothing
  while (maximum(res) > tol && j < k)
    j += 1
    v = A * Q[:, j]
    T[j, j] = Q[:, j]'v
    v = v - T[j, j] * Q[:, j]
    if (j > 1)
      v -= T[j, j - 1] * Q[:, j - 1]
    end
    T[j + 1, j] = norm(v); T[j, j + 1] = T[j + 1, j]
    Q[:, j + 1] = v ./ T[j + 1, j]
    e_j = zeros(Float64, j); e_j[j] = 1.
    f = (T[1:j, 1:j] - sigma * I) \ e_j    
    G = T[1:j, 1:j] + T[j + 1, j]^2 .* f * e_j'
    vals, vecs = eigen(G)
    order = sortperm(norm.(vals .- sigma))[1:minimum([j, nev])]
    vals = vals[order]
    vecs = vecs[:, order]
    rho = [vals[i] - T[j + 1, j]^2 * vecs[:, i]'f * vecs[j, i] for i in 1:minimum([j, nev])]
    res = sqrt.(real.((vals .- rho) .* conj.(rho .- sigma)))
    res ./= norm.(rho)        
    rhos[1:minimum([j, nev]), j] = rho
    println(res)
  end
  return res, rhos[:, 1:j], Q[:, 1:j] * vecs
end;

In [30]:
k = 1_000;
nev = 2;
sigma = 25.;
res, vals, Y = HR_lanczos(A, rand(n), k, nev, sigma, tol);

[2.4530707127185045]
[2.879374493346802, 0.41045990917256536]
[0.5187614768066227, 0.25252210218445664]
[0.22465635872996673, 0.5105030204448706]
[0.15457652485028123, 0.362130387649093]
[0.14750512783646233, 0.29386762132365396]
[0.09469525920371381, 0.2266092270379081]
[0.19646354296214796, 0.10785266012791164]
[0.17825134377403123, 0.08316172938831456]
[0.14344442420728964, 0.10382170006949976]
[0.1361655975687966, 0.08060753119040347]
[0.11841408415987896, 0.09177845072019199]
[0.11433527986204918, 0.06880778887343084]
[0.10350457500111246, 0.08238415950283212]
[0.10050961255642211, 0.06782840368919008]
[0.088782018752989, 0.08203638297779986]
[0.08322091101344309, 0.06442514042204917]
[0.07750915375305607, 0.07444971680465265]
[0.07496944576467272, 0.060926491998757874]
[0.06920510049280246, 0.07929237139688591]
[0.06896054503763004, 0.07281580847099915]
[0.0628028416408093, 0.09714234213416788]
[0.06331371377816224, 0.09107567603488073]
[0.059532260612555135, 0.10200359242893645]

[0.0055641231302265, 0.007584487412853489]
[0.005501702976180473, 0.007588919799895345]
[0.005474871329104748, 0.00753127123790839]
[0.0053772581511827375, 0.007512032393605513]
[0.00536192715038438, 0.007435525850412666]
[0.00526130634923083, 0.007403625692089499]
[0.007346990786032205, 0.005261770942014302]
[0.007252244375308213, 0.00510728672031958]
[0.007131785895947432, 0.005079982477661097]
[0.007048307506207893, 0.004976694209704344]
[0.006938082503084589, 0.004979792913987907]
[0.0068621540084260685, 0.004882073923421631]
[0.006745602453221886, 0.004880390577245232]
[0.006638514807890322, 0.004786224491388466]
[0.006453769716663514, 0.004791250551378168]
[0.006415293951645714, 0.004739710077044987]
[0.00630750868621718, 0.004768679345413795]
[0.006252923417848638, 0.004702286835343329]
[0.006135074240827936, 0.004738317287136989]
[0.006102505606247768, 0.004697109912901234]
[0.005992340213623341, 0.004763847334129532]
[0.005985283833775546, 0.004726134996109016]
[0.005916652827

[0.0036936873685829642, 0.005111389810614027]
[0.0036851728882635565, 0.005042565062839526]
[0.0036679411280561847, 0.004624932246536982]
[0.0036483838966486393, 0.004454799307532799]
[0.0036323507831936197, 0.004205734052287185]
[0.003614297354147779, 0.0040956217967980935]
[0.0036009145578218115, 0.003920900235695507]
[0.0035857416170130747, 0.00384601700346877]
[0.0035638451429269337, 0.0036816991478206456]
[0.0035376908190532476, 0.0035797637855364897]
[0.003527213314985549, 0.0035134623784192063]
[0.0035097123755438784, 0.0034588229524364937]
[0.00349832046375851, 0.0033921270908344623]
[0.003470982286914387, 0.003311707799673337]
[0.0034663575348669227, 0.0032812023798702286]
[0.0034437227426034507, 0.003222841995849315]
[0.0034473344734490515, 0.0032146103114563795]
[0.003431058181622615, 0.003174075368377655]
[0.0034320949944801836, 0.003160847929293401]
[0.0034175869499528454, 0.0031284676267097087]
[0.003427708290878759, 0.003133786140660834]
[0.0034224723635927798, 0.0031227

[0.001177806866728474, 0.002529184188088817]
[0.0011802812844892542, 0.0025286027982661884]
[0.0011784126265077662, 0.002527522113259138]
[0.0011800158145235835, 0.002527252691305303]
[0.001172562266304511, 0.002526835930014896]
[0.0011749656165221537, 0.0025263197691493705]
[0.001168824536289504, 0.002526133613282261]
[0.0011675320128718295, 0.002526258194100919]
[0.0011528467750460075, 0.00252534043398619]
[0.0011541527839890797, 0.0025251284979508704]
[0.0011374387293714453, 0.0025240457835204624]
[0.0011379940640607604, 0.002523992450976942]
[0.001116191213512927, 0.0025213201852366247]
[0.0011168061154348003, 0.0025212773921245006]
[0.0010902152530733015, 0.0025167334315163996]
[0.0010904790060342984, 0.002516598838976343]
[0.0010572032242971725, 0.0025089265258283343]
[0.001053156963375761, 0.0025075835667521342]
[0.001028909678750841, 0.002500186709674297]
[0.0010274580782927312, 0.0024995660623388315]
[0.0010040112152653665, 0.0024910884531388544]
[0.0010021333094355181, 0.0024

[0.0004335596127854738, 0.0022610740972848604]
[0.00042504878138555356, 0.002260011039822674]
[0.00042276686503812857, 0.0022574770722541145]
[0.0004168827077063853, 0.002255929140092716]
[0.00041450398731426256, 0.0022523439607205777]
[0.00040997689214671706, 0.0022495470366053114]
[0.0004065811010045104, 0.0022445203600338303]
[0.00040147651738713075, 0.0022403058352207994]
[0.0004000693706778323, 0.002237432995407344]
[0.0003975886396265348, 0.002235403275975829]
[0.0003953014718940865, 0.002230696770666822]
[0.0003928620138997404, 0.002228287710082588]
[0.000391461557976962, 0.0022248829109809856]
[0.00039061649656887965, 0.0022245215016865817]
[0.0003884635178124552, 0.0022188024486676445]
[0.00038798101894319437, 0.0022185445578784277]
[0.00038501288594258167, 0.0022109513990801063]
[0.00038476322726239926, 0.002211095259151237]
[0.00038146997622878304, 0.002201907343340979]
[0.000381439749836609, 0.0022021003618905867]
[0.0003756332576264139, 0.002185802993105249]
[0.00037517036

[8.603075070615406e-5, 0.00031388433572945406]
[8.54373069181927e-5, 0.00031087759839646046]
[8.057677668609395e-5, 0.0002816066266305369]
[8.045016851710823e-5, 0.0002811479002780704]
[7.397740246712872e-5, 0.00024710993830839314]
[7.337183316306542e-5, 0.0002446759124328585]
[6.873867553137183e-5, 0.0013958794144600325]
[6.832515680246105e-5, 0.0013830996367308866]
[6.505769244411729e-5, 0.0012948033588214762]
[6.483392009297923e-5, 0.0012879553206013967]
[6.199317944773189e-5, 0.001213989947478621]
[6.18550813747428e-5, 0.0012095476216986828]
[5.978490592055647e-5, 0.0011585227024588304]
[5.9784959892280484e-5, 0.0011585461443256824]
[5.766597421917951e-5, 0.0011070182100576115]
[5.762762806509934e-5, 0.0011066741169751268]
[5.6440304120203394e-5, 0.001078504739123607]
[5.563033284346762e-5, 0.0010623467252695975]
[5.324272918998542e-5, 0.001008165985793645]
[5.251463565735348e-5, 0.0009935193577808026]
[5.037268083937126e-5, 0.0009451791026538097]
[4.9459484525617526e-5, 0.00092732

In [31]:
res2 = [norm(A * Y[:, i] - vals[i, end] .* Y[:, i]) / norm(vals[i, end]) for i in 1:nev];
maximum(res - res2)

3.693715865398135e-15

In [32]:
vals[:, end]

2-element Vector{ComplexF64}:
  25.00555991429552 + 0.0im
 25.061552975162282 + 0.0im

In [33]:
Y

7102×2 Matrix{Float64}:
 -0.00166798   -0.00170459
 -0.00334998   -0.00421567
 -0.00610707   -0.010005
  0.00360302    0.00390971
  0.0101474     0.0159739
  0.00257309    0.00371054
 -0.00166761   -0.00151604
 -0.00857561   -0.010363
 -0.012858     -0.0192621
  0.00519499    0.00561935
  0.0140195     0.0192926
  0.00487764    0.00556101
  0.000283372   0.00149261
  ⋮            
  4.84202e-6    6.19084e-6
 -7.45918e-7   -1.12588e-6
 -4.7724e-6    -6.39985e-6
 -1.07311e-6   -1.13519e-6
  3.29972e-7    2.90996e-7
  9.14683e-7    1.04169e-6
  2.67948e-6    4.05669e-6
  1.15438e-8   -3.46597e-8
 -2.61874e-6   -3.68752e-6
 -1.0549e-6    -1.37632e-6
  1.96574e-7    2.4588e-7
  3.83117e-7    4.99403e-7